# Required Libraries

In [79]:
import pandas as pd
import openai
import swifter
from creds import local_chatgpt_api_key
from scipy.spatial.distance import cosine, euclidean
import os
import ast
import numpy as np
import pdb
import time

openai.api_key = local_chatgpt_api_key

# Single prompt

In [7]:
# testing the water
promt_1 = "what is the distance between moon and earth?"

answer = openai.chat.completions.create(
    model = "gpt-4o-mini-2024-07-18", # cheapest model as of the time of scipt
    messages = [{"role": "user", "content": promt_1}])
answer

ChatCompletion(id='chatcmpl-BC9elspuwkKTpohiHRbQFwemTCMDn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The average distance from the Earth to the Moon is about 384,400 kilometers (approximately 238,855 miles). However, this distance can vary slightly due to the Moon's elliptical orbit, ranging from about 363,300 kilometers (225,623 miles) at its closest (perigee) to about 405,500 kilometers (251,966 miles) at its farthest (apogee).", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1742236755, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_06737a9306', usage=CompletionUsage(completion_tokens=84, prompt_tokens=16, total_tokens=100, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetai

In [8]:
answer.choices[0].message.content

"The average distance from the Earth to the Moon is about 384,400 kilometers (approximately 238,855 miles). However, this distance can vary slightly due to the Moon's elliptical orbit, ranging from about 363,300 kilometers (225,623 miles) at its closest (perigee) to about 405,500 kilometers (251,966 miles) at its farthest (apogee)."

# Multiple prompt

In [12]:
# testing the water part 2
promt_2 = "you are a assistant who is helping me answer question. Please answer in a poem"

answer = openai.chat.completions.create(
    model = "gpt-4o-mini-2024-07-18", # cheapest model as of the time of scipt
    messages = [{"role": "system", "content": promt_2},
    {"role": "user", "content": promt_1}])
answer

ChatCompletion(id='chatcmpl-BCJQfNwF6KdAlzXpkeKogPJt8EE5V', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='In the night sky, a traveler gleams,  \nThe Moon, with its glow, inspires our dreams.  \nFrom Earth, it hangs, a silvery sphere,  \nAbout two hundred fifty thousand miles, not so near.  \n\nA dance in the darkness, a waltz of delight,  \nA journey through cosmos, from day into night.  \nSo when you gaze up and see it so bright,  \nRemember that distance, in the stillness of night.  ', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1742274321, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_06737a9306', usage=CompletionUsage(completion_tokens=97, prompt_tokens=36, total_tokens=133, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_predicti

In [14]:
print(answer.choices[0].message.content)

In the night sky, a traveler gleams,  
The Moon, with its glow, inspires our dreams.  
From Earth, it hangs, a silvery sphere,  
About two hundred fifty thousand miles, not so near.  

A dance in the darkness, a waltz of delight,  
A journey through cosmos, from day into night.  
So when you gaze up and see it so bright,  
Remember that distance, in the stillness of night.  


# Getting Webpage contents

## Scrape the Website and Split the Content

In [155]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

loader = WebBaseLoader("https://www.newegg.ca/p/pl?N=100006655&PageSize=96")
# loader = WebBaseLoader("https://www.ml.school/")
documents = loader.load_and_split(text_splitter)
documents

[Document(metadata={'source': 'https://www.newegg.ca/p/pl?N=100006655&PageSize=96', 'title': 'Motherboard | Newegg.ca', 'description': 'Shop Motherboard on Newegg.ca. Watch for amazing deals and get great pricing.', 'language': 'en-ca'}, page_content='Motherboard | Newegg.ca'),
 Document(metadata={'source': 'https://www.newegg.ca/p/pl?N=100006655&PageSize=96', 'title': 'Motherboard | Newegg.ca', 'description': 'Shop Motherboard on Newegg.ca. Watch for amazing deals and get great pricing.', 'language': 'en-ca'}, page_content="Hello\xa0Select address\xa0SearchChange CountryReturns& OrdersMenu Shell Shocker DealsClearance DealsBest SellersElite DealsPC BuilderSell on NeweggCombo DealsGaming PC FinderNAS BuilderASUS NUC ConfiguratorNewegg BusinessFeedbackHelp CenterYour Browsing HistoryHome Components & Storage Motherboard Motherboard\xa0Save this search Remove Combo Deal OptionsONOFFIn StockONOFFSold by NeweggONOFFSubCategoryIntel MotherboardsAMD MotherboardsIoT Edge Device / Development 

## Create a Knowledge Base

In [156]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df#head(10)

,text
0,Motherboard | Newegg.ca
1,Hello Select address SearchChange CountryRetur...
2,moreShow MoreCPU Socket TypeLGA 1851LGA 1851LG...
3,i7 / i5 / i3 (LGA1151)Core i7 / i5 / i3 (8th G...
4,H110Intel 9 SeriesIntel X99Intel H97Intel 8 Se...
5,(DDR5)4x288pin (DDR4)4x260pin4x240pin2 Slots2x...
6,USB 3.04 x USB 3.1 Gen 14 x USB 3.1 Gen 1 Type...
7,x Thunderbolt 31 x Thunderbolt 42 x Thunderbol...
8,(Support PCIe 3.0 x4)PCIe & SATA M.21 x M.2 (S...
9,+ 1Gbps5Gbps + 2.5Gbps5Gbps + 1Gbps10Gbps + 1G...


## Embedding with openai 

In [157]:
def embed_via_openai_model(text):
    model = 'text-embedding-3-small'
    text = text.replace("\n", " ")
    return openai.embeddings.create(input = [text], model = model).data[0].embedding

In [159]:
embed_via_openai_model(df['text'].iloc[0])

[0.041252363473176956,
 0.011405707336962223,
 -0.0012732023606076837,
 -0.008461711928248405,
 -0.0339682511985302,
 -0.00245687086135149,
 -0.022774994373321533,
 0.025761481374502182,
 -0.021755218505859375,
 0.007824352011084557,
 0.04516150429844856,
 0.038023073226213455,
 0.01597648859024048,
 -0.0018316510831937194,
 0.008431361056864262,
 0.014993133954703808,
 -0.0005186137277632952,
 0.020796144381165504,
 -0.02481454610824585,
 0.03156449273228645,
 0.04183509200811386,
 0.02598000504076481,
 0.045598551630973816,
 -0.023212041705846786,
 0.020638322457671165,
 0.008437431417405605,
 0.0016374080441892147,
 -0.017166227102279663,
 0.017251208424568176,
 -0.02440178021788597,
 0.05016326159238815,
 -0.02692694030702114,
 -0.016595639288425446,
 0.004892496392130852,
 -0.052785541862249374,
 -0.054242365062236786,
 -0.055359262973070145,
 -0.03855724260210991,
 -0.025688640773296356,
 0.008904828689992428,
 0.022022303193807602,
 0.020832564681768417,
 0.011642441153526306,
 

In [160]:
%%time

df['embeddings'] = df['text'].apply(embed_via_openai_model)

CPU times: user 212 ms, sys: 82 ms, total: 294 ms
Wall time: 31.7 s


In [162]:
df.to_csv('chunck_1000_newegg_webscrape_embedding.csv', index = False)
df.to_pickle('chunck_1000_newegg_webscrape_embedding.pkl')

In [163]:
df = pd.read_pickle("chunck_1000_newegg_webscrape_embedding.pkl")
df

,text,embeddings
0,Motherboard | Newegg.ca,"[0.041252363473176956, 0.011405707336962223, -..."
1,Hello Select address SearchChange CountryRetur...,"[0.017726359888911247, -0.015840880572795868, ..."
2,moreShow MoreCPU Socket TypeLGA 1851LGA 1851LG...,"[0.005272025242447853, -0.007769968826323748, ..."
3,i7 / i5 / i3 (LGA1151)Core i7 / i5 / i3 (8th G...,"[0.035840000957250595, 0.007440057583153248, 0..."
4,H110Intel 9 SeriesIntel X99Intel H97Intel 8 Se...,"[0.01975996419787407, -0.03057100996375084, 0...."
5,(DDR5)4x288pin (DDR4)4x260pin4x240pin2 Slots2x...,"[0.02917781099677086, -0.023983579128980637, 0..."
6,USB 3.04 x USB 3.1 Gen 14 x USB 3.1 Gen 1 Type...,"[-0.012260204181075096, -0.013470755890011787,..."
7,x Thunderbolt 31 x Thunderbolt 42 x Thunderbol...,"[0.03886720910668373, -0.013168738223612309, 0..."
8,(Support PCIe 3.0 x4)PCIe & SATA M.21 x M.2 (S...,"[0.04627262055873871, -0.019628623500466347, 0..."
9,+ 1Gbps5Gbps + 2.5Gbps5Gbps + 1Gbps10Gbps + 1G...,"[0.01419492345303297, 0.02637331932783127, -0...."


In [164]:
prompt = "which is the cheapest AM5 motherboard"
embeded_prompt = embed_via_openai_model(prompt)

In [165]:
embeded_prompt

[0.020031379535794258,
 0.00902564823627472,
 -0.0038720101583749056,
 0.0056622214615345,
 -0.004678961355239153,
 0.0015138813760131598,
 0.011833025142550468,
 0.021048545837402344,
 -0.005624925252050161,
 -0.017698679119348526,
 0.02427634969353676,
 -0.022119959816336632,
 -0.04133760556578636,
 0.0396016426384449,
 0.01805129647254944,
 -0.02967410907149315,
 -0.007038785144686699,
 0.004923081025481224,
 -0.029809730127453804,
 -0.0014358987100422382,
 0.0481458343565464,
 0.03471925109624863,
 -0.0032583202701061964,
 0.017427435144782066,
 0.009113802574574947,
 -0.02490021102130413,
 0.01016487367451191,
 -0.04478240758180618,
 -0.028100892901420593,
 -0.010225904174149036,
 0.05289260670542717,
 -0.04638274759054184,
 0.025225704535841942,
 0.0007789792725816369,
 -0.04603013023734093,
 0.03216955438256264,
 0.002763299737125635,
 -0.051889002323150635,
 -0.013806326314806938,
 -0.019366830587387085,
 0.02942998893558979,
 0.009710540063679218,
 0.020885799080133438,
 0.012

In [166]:
def dot(page_embedding):
    dot = np.dot(page_embedding, embeded_prompt)
    return dot

def Cosine_similarity(page_embedding):
    # cosine_similarity = np.dot(page_embedding, embeded_prompt) / (np.linalg.norm(page_embedding) * np.linalg.norm(embeded_prompt))
    cosine_similarity = 1 - cosine(page_embedding, embeded_prompt)
    return cosine_similarity

def euclidean_distance(page_embedding):
    # euclidean_distance = np.linalg.norm(np.array(page_embedding) - np.array(embeded_prompt))
    euclidean_distance = euclidean(page_embedding, embeded_prompt)
    return euclidean_distance

# df['dot_relevance'] = df['embeddings'].swifter.apply(dot)
df['cosine_similarity'] = df['embeddings'].swifter.apply(Cosine_similarity)
df['euclidean_distance'] = df['embeddings'].swifter.apply(euclidean_distance)

Pandas Apply:   0%|          | 0/39 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/39 [00:00<?, ?it/s]

# Cosine

In [171]:
df.sort_values('cosine_similarity', ascending = False, inplace = True)
df

,text,embeddings,cosine_similarity,euclidean_distance
36,ShippingAdd to cart CompareQuick View(699)ASRo...,"[0.020336762070655823, -0.009349689818918705, ...",0.606330,0.887322
11,options from $249.99 - $433.97Free ShippingAdd...,"[0.01952257752418518, -0.008336021564900875, -...",0.587150,0.908681
17,SATA 6G 2x M.2 2.5G LAN 7.1 Audio mATX Motherb...,"[0.015331065282225609, 0.006068546790629625, -...",0.572134,0.925058
19,"8000 & 7000 ATX motherboard, DDR5, PCIe 5.0 M....","[0.024009590968489647, -0.001491393311880529, ...",0.567129,0.930453
35,View(502)ASUS ROG STRIX X670E-F GAMING WIFI AM...,"[0.011290728114545345, -0.0041331020183861256,...",0.562049,0.935897
22,B850 GAMING PLUS WIFI AM5 AMD B850 SATA 6Gb/s ...,"[0.013946790248155594, -0.004968107677996159, ...",0.557743,0.940486
20,options from $129.99 - $159.00Free ShippingAdd...,"[-0.0035412912257015705, -0.003833578433841467...",0.555918,0.942425
18,MAG X670E TOMAHAWK WIFI$359.99 $314.99 –Save: ...,"[0.01011571753770113, -0.025029310956597328, -...",0.555169,0.943219
10,RatingMost ReviewsBest Selling View:36609696 ...,"[0.020226752385497093, 0.0029305447824299335, ...",0.555102,0.943290
29,#: PRIME B550M-A AC$159.99 $149.99 –Save: $10....,"[0.01458571758121252, 0.00289578246884048, -0....",0.551793,0.946792


In [172]:
context = df['text'].iloc[0] + '\n' + df['text'].iloc[1] + '\n' + df['text'].iloc[2] + '\n' + df['text'].iloc[3] + '\n' + df['text'].iloc[4] + '\n'
print(context)

ShippingAdd to cart CompareQuick View(699)ASRock B650E TAICHI AM5 EATX Motherboard, 4 slots DDR5, PCIE 5.0 x16,  3 M.2 slots,  2.5Gb Lan,WIFI 6E,  HDMI 2.1 port, 5.1 audio ALC4082, 2 USB4 Thunderbolt Type-C  24+2+1 Power Phase, 8 layer PCBModel #: B650E TAICHI $429.99 –Free ShippingAdd to cart CompareQuick View(346)GIGABYTE X870I AORUS PRO ICE AMD AM5 LGA 1718 Motherboard, Mini-ITX, DDR5, 2x M.2, PCIe 5.0, USB-C, WIFI7, 2.5GbE LAN, EZ-Latch, 5-Year WarrantyModel #: X870I AORUS PRO ICE $439.99 –More options from $439.99 - $520.54$10.17 ShippingAdd to cart CompareQuick View(699)ASRock B650M-HDV/M.2 Socket AM5 Ryzen 7000 Micro ATX MotherboardModel #: B650M-HDV/M.2 $232.00 –More options from $190.44 - $293.82Free ShippingAdd to cart CompareQuick View(636)MSI PRO B650M-A WIFI AM5 AMD B650 SATA 6Gb/s DDR5 Ryzen 7000 Micro ATX MotherboardModel #: PRO B650M-A WIFI$239.99 $214.99 –Save: $25.00 (10%)More options from $214.99 - $302.99Free ShippingAdd to cart CompareQuick View(760)ASUS Prime
opti

In [173]:
response = openai.chat.completions.create(
    model = "gpt-4o-mini-2024-07-18",
    messages = [
    {"role": "system", "content": "you are at customer service for newegg to help customers with their purchase"},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": f"use this information from from the newegg\'s motherboard page to help customer\'s question with this context: {context}. Please make a proper and polite context"}
    ])

In [174]:
# cosine
print(response.choices[0].message.content)

Hello! Thank you for reaching out. 

If you're looking for the cheapest AM5 motherboard available on Newegg, the **ASRock B650M Pro RS WiFi** is currently priced at **$214.99**. It features SATA 6Gb/s and is a Micro ATX motherboard which can be a great option if you're on a budget.

If you have any more questions or need assistance with anything else, feel free to ask!


# Euclidean

In [175]:
df.sort_values('euclidean_distance', ascending = False, inplace = True)
df

,text,embeddings,cosine_similarity,euclidean_distance
38,camera.Customer ServiceHelp CenterTrack an Ord...,"[0.015055366791784763, -0.001766583533026278, ...",0.290718,1.191035
6,USB 3.04 x USB 3.1 Gen 14 x USB 3.1 Gen 1 Type...,"[-0.012260204181075096, -0.013470755890011787,...",0.298388,1.184578
5,(DDR5)4x288pin (DDR4)4x260pin4x240pin2 Slots2x...,"[0.02917781099677086, -0.023983579128980637, 0...",0.333129,1.154877
8,(Support PCIe 3.0 x4)PCIe & SATA M.21 x M.2 (S...,"[0.04627262055873871, -0.019628623500466347, 0...",0.339867,1.149028
7,x Thunderbolt 31 x Thunderbolt 42 x Thunderbol...,"[0.03886720910668373, -0.013168738223612309, 0...",0.364180,1.127670
3,i7 / i5 / i3 (LGA1151)Core i7 / i5 / i3 (8th G...,"[0.035840000957250595, 0.007440057583153248, 0...",0.379819,1.113715
2,moreShow MoreCPU Socket TypeLGA 1851LGA 1851LG...,"[0.005272025242447853, -0.007769968826323748, ...",0.382621,1.111196
9,+ 1Gbps5Gbps + 2.5Gbps5Gbps + 1Gbps10Gbps + 1G...,"[0.01419492345303297, 0.02637331932783127, -0....",0.418409,1.078510
4,H110Intel 9 SeriesIntel X99Intel H97Intel 8 Se...,"[0.01975996419787407, -0.03057100996375084, 0....",0.430516,1.067225
0,Motherboard | Newegg.ca,"[0.041252363473176956, 0.011405707336962223, -...",0.479423,1.020369


In [176]:
context = df['text'].iloc[0] + '\n' + df['text'].iloc[1] + '\n' + df['text'].iloc[2] + '\n' + df['text'].iloc[3] + '\n' + df['text'].iloc[4] + '\n'
print(context)

camera.Customer ServiceHelp CenterTrack an OrderReturn an ItemReturn PolicyPrivacy & SecurityFeedbackMy AccountLogin/RegisterOrder HistoryReturns HistoryAddress BookWish ListsMy Build ListsEmail NotificationsSubscriptions OrdersAuto NotificationsCompany InformationAbout NeweggAwards/RankingsHours and LocationsPress InquiriesCareersNewsroomNewegg BlogTools & ResourcesSell on NeweggShipped by Newegg3PL LogisticsBecome an AffiliateBecome a Newegg VendorSite MapRebatesGift CardsMobile AppsNewegg Media ServiceShop Our BrandsNewegg GlobalABSRosewill  55 East Beaver Creek Road, Unit E and F, Richmond Hill, Ontario, L4B 1E8newsletter@newegg.caPolicy & AgreementPrivacy Policy © 2000-2025 Newegg Inc  As used herein, “Newegg.ca” means Newegg Canada Inc. unless the context specifically refers to the  www.newegg.ca  web address.FacebookTwitterInstagramLinkedInPinterestYouTubeTwitchDiscordTikTocCompare Items123456789101112131415COMPAREClear AllMessages
USB 3.04 x USB 3.1 Gen 14 x USB 3.1 Gen 1 Type-

In [173]:
response = openai.chat.completions.create(
    model = "gpt-4o-mini-2024-07-18",
    messages = [
    {"role": "system", "content": "you are at customer service for newegg to help customers with their purchase"},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": f"use this information from from the newegg\'s motherboard page to help customer\'s question with this context: {context}. Please make a proper and polite context"}
    ])

In [170]:
# euclidean
print(response.choices[0].message.content)

Thank you for reaching out! To help you find the cheapest AM5 motherboard available, I recommend checking the motherboard section on Newegg's website, where you can filter options by price and type. New products and promotional discounts are frequently offered, so this will allow you to find the best deal.

If you need assistance navigating the site or have any specific requirements for your motherboard, please let me know! I'm here to help.
